In [6]:
import numpy as np 
import matplotlib.pyplot as plt
from qdc.grating.grating_sim import GratingSim1D

In [111]:
sim = GratingSim1D(
    Nx          = 2**14,      # grid points
    Lx          = 8e-3,       # 8 mm
    wl0         = 808e-9,     # 808 nm blaze
    Dwl         = 80e-9,     # ±50 nm span
    N_wl        = 101,         # 11 wavelengths  (odd!)
    waist       = 1000e-6,      # 40 µm beam waist
    x0          = 0,
    blaze_angle = 0.1,       # blaze angle [rad]
    f           = 0.2,         # lens focal length [m]
    spectrum='flat', 
)

x_det, I_class = sim.classical_pattern()
_,     I_spdc  = sim.spdc_pattern()
orders = sim.diffraction_orders(x_det)
# after you compute I_class and I_spdc and orders:
epsilon = (sim.wls.max() - sim.wls.min())/(2*sim.wl0)

eta_class = epsilon**2
eta_spdc  = epsilon**4

fig, ax = plt.subplots(figsize=(7,4))
ax.semilogy(orders, I_class, label="classical")
ax.semilogy(orders, I_spdc,  label="SPDC")

# draw analytic guides at m=2 and m=3
ax.hlines(eta_class, 1.9, 2.1, color='C0', linestyle='--', label=r'$\epsilon^2$')
ax.hlines(eta_spdc,  2.9, 3.1, color='C1', linestyle='--', label=r'$\epsilon^4$')

# ax.set_xlim(0.5, 3.5)
ax.set_xlabel("diffraction order $m$")
ax.set_ylabel("normalized intensity (log)")
ax.legend()
fig.show()

In [73]:
from qdc.grating.grating_sim import gaussian, blazed_phase

if True:
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(np.angle(sim.grating_mask), '--.')
    # ax.set_xlim([0, 1400])
    fig.show()
 
if True:
    E = gaussian(sim.x, sim.waist, sim.x0).astype(np.complex128)  # <<<
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot((np.abs(E))**2)
    fig.show()